# flavio tutorial

## Part 4: Likelihoods

## "Likelihoods" in flavio

A selection of *observables* and their *measurements* defines a likelihood in the space of a selection of *parameters* or *Wilson coefficients*

Example: The SMEFT likelihood implemented in `smelli` considers a large set of around 400 observables and their measurements and is defined in the space of dimension six SMEFT Wilson coefficients

## The likelihood function

$$\mathcal L (\vec C, \vec\theta)=\prod_{i=1} \mathcal L^i_\text{exp}\!\left(O_i=O_i^\text{th}(\vec C, \vec\theta)\right)\times \mathcal L_{\theta}(\vec \theta) $$

where
- $\mathcal L^\text{exp}(O_i)$ probability distribution functions of experimental measurements of observables $O_i$
- $O_i^\text{th}(\vec C, \vec \theta)$ theory predictions for observables
- $\vec C$ set of WET or SMEFT Wilson coefficients
- $\vec \theta$ set of parameters
- $\mathcal L_\theta(\vec \theta)$ experimental and theoretical constraints on parameters

- The parameters $\vec \theta$ can be split into fit parameters $\vec \xi$ and nuisance parameters $\vec \nu$
- For a likelihood in Wilson coefficient space, all parameters $\vec \theta$ are nuisance parameters


Different ways to obtain "nuisance-free" likelihood

- **Bayesian fit**: treat uncertainties on nuisance parameters as prior probabilities and use Bayes' theorem to *marginalize* over them (multi-dimensional integration requiring tools like Markov Chain Monte Carlo)

- **Frequentist fit**: treat uncertainties on nuisance parameters as pseudo measurements and *profile* over them, i.e. optimizing the likelihood in nuisance parameter space for each point in fit parameter space

This can require a lot of computing power and time... 

Approximations:

- **Fast likelihood**: convolve the experimental uncertainties with the theoretical ones to construct a likelihood that only depends on fit parameters

- **Very small theory uncertainty**: dependence of $\mathcal L_\theta$ on nuisance parameters can be neglected

## Fast Likelihood

Assumptions & advantages:
- (!) experimental uncertainties approximated as Gaussian
- (!) theory uncertainties approximated as Gaussian
- (!) uncertainties weakly dependent on fit parameters
- (+) Computing time does not depend on number of nuisance parameters
- (+) Computation of covariances (expensive) does not depend on exp. data

## Setting up a "Fast Likelihood"

Example: $C_7$ and $C_{10}$ from $B\to X_s\gamma$ and $B_s\to\mu^+\mu^-$

In [ ]:
import flavio
from flavio.statistics.likelihood import FastLikelihood

In [ ]:
FL = FastLikelihood(name='My fast likelihood',
                    observables=['BR(B->Xsgamma)', 'BR(Bs->mumu)', 'BR(B0->mumu)'],
                   )

NB: `BR(B0->mumu)` is not sensitive to these coefficients but must be added since the measurements are correlated. Try running the command without it!

Let's have a look at the measurements available for our observables.

In [ ]:
set(
    flavio.Observable['BR(B->Xsgamma)'].get_measurements() +
    flavio.Observable['BR(Bs->mumu)'].get_measurements() +
    flavio.Observable['BR(B0->mumu)'].get_measurements()
)

But we don't want to include *both* a combination and the individual measurements!

What is the problem?

By default, `FastLikelihood` uses all measurements available for a given observable.

Solution: We have to either exclude unwanted measurements or include only those we want.

In [ ]:
FL = FastLikelihood(name='My fast likelihood',
                    observables=['BR(B->Xsgamma)', 'BR(Bs->mumu)', 'BR(B0->mumu)'],
                    include_measurements=['ATLAS CMS LHCb combination Bs->mumu 2020 Gaussian', 'B->Xgamma WA 2017'])

### A word of caution

This is a general pitfall: when constructing a likelihood, one has to be careful to include observables and measurements consistently.

A consitent likelihood based on `flavio` is already included in the package `smelli`:

[github.com/smelli/smelli](https://github.com/smelli/smelli)

### Computing the covariance

To compute the covariance of the "pseudo measurement" with combined experimental and theoretical uncertainties, need to call:

In [ ]:
%time FL.make_measurement(N=100)

Advanced: increase precision, use parallelization

In [ ]:
%time FL.make_measurement(N=1000, threads=8, force=True)  # force recomputation

Now we define a function for plotting and/or minimization

In [ ]:
from wilson import Wilson

In [ ]:
par = flavio.parameters.default_parameters.get_central_all()

def FLL(x):
    Re_C7, Re_C10 = x
    w = Wilson({'C10_bsmumu': Re_C10, 'C7_bs': Re_C7},
                scale=4.8,
                eft='WET', basis='flavio')
    return FL.log_likelihood(par, w)

### Minimize the chi2

In [ ]:
from flavio.math.optimize import minimize

In [ ]:
def chi2(x):
    return -2*FLL(x)

In [ ]:
chi2_min = minimize(chi2,[0,0])
chi2_min

### 2D likelihood plots

In [ ]:
import matplotlib.pyplot as plt
import flavio.plots as fpl

In [ ]:
%%time
cdat = fpl.likelihood_contour_data(FLL,
                                   x_min=-0.07, x_max=0.08, y_min=-2, y_max=2,
                                   n_sigma=(1, 2, 3),
                                   threads=8,
                                   steps=20)

In [ ]:
fpl.contour(**cdat, z_min=chi2_min['fun'])
plt.xlabel(r'Re $C_7$');
plt.ylabel(r'Re $C_{10}$');

This can be made more smooth by increasing `steps` or using interpolation (but be careful...)

In [ ]:
fpl.contour(**cdat, z_min=chi2_min['fun'], interpolation_factor=3)
plt.xlabel(r'Re $C_7$');
plt.ylabel(r'Re $C_{10}$');

## Neglecting theory uncertainties

In [ ]:
from flavio.statistics.likelihood import Likelihood

In [ ]:
L = Likelihood(observables=['BR(B->Xsgamma)', 'BR(Bs->mumu)', 'BR(B0->mumu)'],
               include_measurements=['inofficial combination Bs->mumu 2019 CMS19', 'B->Xgamma WA 2017'])

In [ ]:
par = flavio.parameters.default_parameters.get_central_all()

def LL(x):
    Re_C7, Re_C10 = x
    w = Wilson({'C10_bsmumu': Re_C10, 'C7_bs': Re_C7},
                scale=4.8,
                eft='WET', basis='flavio')
    return L.log_likelihood(par, w)

### Minimize the chi2

In [ ]:
def chi2_LL(x):
    return -2*LL(x)

In [ ]:
chi2_min_LL = minimize(chi2_LL,[0,0])
chi2_min_LL

### 2D likelihood plots

In [ ]:
%%time
cdat = fpl.likelihood_contour_data(LL,
                                   x_min=-0.07, x_max=0.08, y_min=-2, y_max=2,
                                   n_sigma=(1, 2, 3),
                                   threads=8,
                                   steps=20)

In [ ]:
fpl.contour(**cdat, z_min=chi2_min_LL['fun'], interpolation_factor=10)
plt.xlabel(r'Re $C_7$');
plt.ylabel(r'Re $C_{10}$');

Result in $C_7$ direction is not acceptable since theory uncertainties cannot be neglected...

Next: <a href="../smelli-talk/smelli.ipynb">smelli</a>